<a href="https://colab.research.google.com/github/Sotdo/Variant_Calling/blob/master/depletioniTimeCalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import glob
import sys
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print("""
***************************************************
Script Descriptioin
===================================================
Calculate the depletion time according to the following paper:

https://www.pnas.org/content/117/30/18010.full
Ranking essential bacterial processes by speed of mutant deat
PNAS July 28, 2020 117 (30) 18010-18017; first published July 14, 2020
===================================================
""")



***************************************************
Script Descriptioin
Calculate the depletion time according to the following paper:

https://www.pnas.org/content/117/30/18010.full
Ranking essential bacterial processes by speed of mutant deat
PNAS July 28, 2020 117 (30) 18010-18017; first published July 14, 2020



 # 1. Method of depletion time calculation in the literature
 **source code**
 ```R
 locus_depl_time <- function(times, log_reads, thresh = depl_thresh) {
   # get the number of the last lrpk value above the threshold
   aboves <- which(log_reads >= thresh)
   last_above <- last(aboves)
   # return NaN if it's the last time point
   if (last_above == length(log_reads)) { return( NaN ) }
   # calculate the depletion time
   dt <- times[last_above] + ( times[last_above + 1] - times[last_above] ) *
     ( thresh - log_reads[last_above] ) / ( log_reads[last_above + 1] - log_reads[last_above] )
   return(dt) }
 ```
 假设这里有四个时间点 [t0,t1,t2,t3] 和对应的 reads 数目的 Fold change [f0,f1,f2,f3],其中 f0 应该为 0。
 然后判断 -1 落在 [f0,f1,f2,f3] 哪个区间中，如果 [f0,f1,f2,f3] 是单调递减的，那么 -1 只会落在一个区间里，比如落在
 [f1,f2] 这个区间，那么用 (t1,f1) 和 (t2,f2) 这两个点做线性插值，即线性拟合后计算 Fold change 为 -1 的时候的时间
 即为 depletion time。
 如果 [f0,f1,f2,f3] 是波动变化的，-1 可能落在多个区间，这种情况作者选择时间点最后的 -1 落在的区间做线性插值来计算
 depletion time。根据观察，这样的情况在该文献中出现的情况很少，约 ~500 个里面出现 ～5 左右。

 # 2. Use the same method to calculate the depletion time in our data
 我们数据中的时间点: [Spore, YES1, YES2, YES3, YES4], 为了方便计算时间点的值分别计为 [0,1,2,3,4]
 根据 Normalized read count 计算 log2(Fold change) 然后计算 depletion time。
 # 3. Import the 1328_400G data (data from SF)

In [4]:
inputFile = Path("/content/20210719_1328_400G_sites_MA_normalized.csv")
sites_400G =  pd.read_csv(inputFile,sep="\t",header=0)

# drop the arg6 site
# sites_400G.sort_values(["NSK-1328-1328_36h_spore"],ascending=False,inplace=True)
# sites_400G_noarg6 = sites_400G.iloc[1:,:]

outputFolder = Path("depletion_time_calculation")
outputFolder.mkdir(exist_ok=True)


 # 4. Calculate the fold change.
 log2(Fold change) = log2(YESX+1) - log2(Spore+1)

In [5]:
timePoints = [
    'NSK-1328-1328_36h_spore',
    'NSK-1328-1328_36h_YES1',
    'NSK-1328-1328_36h_YES2',
    'NSK-1328-1328_36h_YES3',
    'NSK-1328-1328_36h_YES4'
]
for t in timePoints:
    sites_400G["log2FC_{}".format(t)] = np.log2(sites_400G[t]+1) - np.log2(sites_400G["NSK-1328-1328_36h_spore"]+1)

 # 5. Calculate the depletion time
 我们的数据比较复杂，Fold change 可能有 [0,-0.7,-1.3,-1.1,-0.9] 这种情况，那应该用哪个 -1 所在的区间来计算 depletion time 呢？
 下面计算用的区间是后一个 -1 所在的递减区间来计算的。

In [6]:
def cal_depletion_time(FC):
    # FC = row["log2FC_NSK-1328-1328_36h_spore":"log2FC_NSK-1328-1328_36h_YES4"]
    try:
        lt_Positions = np.where(FC<-1)[0]
        depletion_times = []
        for i in lt_Positions:
            gt_Position = i - 1
            if FC[gt_Position] > -1 and FC[i] < -1:
                depletion_times.append((-1-FC[gt_Position])/(FC[i] - FC[gt_Position])+gt_Position)
        # depletion_time = (-1-FC[gt_Position])/(FC[i] - FC[gt_Position])+gt_Position
        return(depletion_times[len(depletion_times)-1])
    except (ValueError, IndexError):
        return(np.nan)

sites_400G["Depletion_Time"] = sites_400G.apply(lambda row: cal_depletion_time(row["log2FC_NSK-1328-1328_36h_spore":"log2FC_NSK-1328-1328_36h_YES4"]),axis=1)

file_withDepletionTime = outputFolder / Path("20210719_1328_400G_sites_MA_normalized_withDepletionTime.xlsx")
sites_400G.to_excel(file_withDepletionTime)

 # 6. Sites with YES4.M > 2

In [ ]:
YES4M_cutoff_2 = sites_400G["NSK-1328-1328_36h_YES4.M"] >= 2

sites_YES4M_cutoff_2 = outputFolder / Path("20210719_1328_400G_sites_MA_normalized_withDepletionTime_YES4Mgt2.xlsx")
sites_400G[YES4M_cutoff_2].to_excel(sites_YES4M_cutoff_2)


 # 7. Overview of sites with YES4.M > 2

In [ ]:
sns.set_theme(style="darkgrid")
fig1,ax1 = plt.subplots(1,4,figsize=(24,24),sharey=True)
for i in range(4):
    # sns.boxplot(data=sites_400G_noarg6,x="Type_Interval",y="NSK-1328-1328_36h_YES{}.M".format(i+1),ax=ax1[i])
    sns.violinplot(data=sites_400G[YES4M_cutoff_2],x="Type_Interval",y="NSK-1328-1328_36h_YES{}.M".format(i+1),ax=ax1[i])
    ax1[i].set_xticklabels(ax1[i].get_xticklabels(),rotation=75,fontsize=15)
    ax1[i].set_xlabel(ax1[i].get_xlabel(),fontsize=18)
    # ax1[i].set_yticklabels(ax1[i].get_yticklabels(),fontsize=20)
    ax1[i].set_ylim(-7,9)
    ax1[i].set_ylabel(ax1[i].get_ylabel(),fontsize=18)
    ax1[i].set_title("YES{} M".format(i+1),fontsize=25)
# plt.savefig(outputFolder/Path("Violinplot_M_Type_interval.png"),bbox_inche="tight")

In [ ]:
sns.boxplot(data=sites_400G[YES4M_cutoff_2],x="Type_Interval",y="Depletion_Time")